In [3]:
import json
from mention import *
from collections import Counter
from math import log
from gensim.matutils import argsort
from gensim.corpora import Dictionary
from gensim.models import Word2Vec
import numpy as np
import random
import sys
from random import randint
from data_load import *
from fast_read import *
from cwsabie_label import save2bin
def total_count(d):
    return reduce(lambda x,y:x+y,d.values(),0)
def lambda_p(l,e,labels,entities,label_entity,t_l):
    if "%s_%s" % (l,e) not in label_entity: return -1.
    c = log(t_l)
    return (log(labels[l]) + log(entities[e]) - 2*c)/(log(label_entity["%s_%s" % (l,e)])-c) -1
def proto(l,es,labels,entities,label_entity,t_l,topn=10):
    scores = [lambda_p(l,e,labels,entities,label_entity,t_l) for e in es]
    return [es[i] for i in argsort(scores,topn=topn,reverse=True)]

# PREPARE STATISTIC FROM FILES

In [2]:
stop = set([ ln.rstrip() for ln in open('stop.list')])
filename = '/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_new.json'
def pre_pmi(train_file):
    relation_id = dict()
    label_id = dict()
    words = set()
    labels = Counter()
    entities = Counter()
    label_entity = Counter()
    with open(filename,'r') as f:
            for line in f:
                sent = json.loads(line.strip('\r\n'))

                mentions = [Mention(int(m['start']), int(m['end']), m['labels']," ".join(sent['tokens'][m['start']:m['end']]),\
                                    mid='%s_%d_%d_%d'%(sent['fileid'], sent['senid'], m['start'], m['end'])) for m in sent['mentions']]
                for m in mentions:
                    for l in m.labels:
                       # if m.name.lower() in stop:continue
                        for e in [m.name.lower()]:#.split():
                            if e not in stop:
                                labels[l]+=1
                                entities[e]+=1
                                label_entity["%s_%s" % (l,e)]+=1
    return labels,entities,label_entity

# COMPUTE NPMI AS PROTOTYPES OF LABELS

In [23]:
def get_proto_embedding_avg(fn,labels,entities,label_entity,topn=55):
    #model = Word2Vec.load_word2vec_format(fn)
    vocab,id2word,embedding = readEmbed(fn)
    t_l = total_count(labels)
    size = embedding.shape[1]
    es = [e for e in entities]
    label_ebd = np.zeros([len(labels),size])
  #  lout.write("%d %d\n" % (len(labels),300))
    for i,l in enumerate(labels):
        protoes = proto(l,es,labels,entities,label_entity,t_l,topn=topn)
        avg = np.zeros(size,dtype=np.float32)
        hit = 0
        
        for w in protoes:
            w = w.split()[-1]
            if w in vocab and w not in stop:
                hit+=1
                avg += embedding[vocab[w]]

        avg /= hit
        text =  " ".join(map(str,avg))
        label_ebd[i] = avg
    return label_ebd
def get_proto_embedding_lst(fn,labels,entities,label_entity,topn=55):
    #model = Word2Vec.load_word2vec_format(fn)
    vocab,id2word,embedding = readEmbed(fn)
    t_l = total_count(labels)
    size = embedding.shape[1]
    es = [e for e in entities]
    label_ebd = []

    for i,l in enumerate(labels):
        protoes = proto(l,es,labels,entities,label_entity,t_l,topn=topn)
        avg = []
        hit = 0
        words =  set([w.split()[-1] for w in protoes if w.split()[-1] in vocab and w not in stop] )
        for w in words:
            if w in vocab and w not in stop:
                hit+=1
                avg.append(w)
                #avg.append(embedding[vocab[w]])
#         if hit ==0:
#             print l,"shit"
            #print l,protoes   
        label_ebd.append(avg)
    return label_ebd

In [34]:
labels,entities,label_entity = pre_pmi(filename)
label_ebd=  get_proto_embedding_lst('../../bow2.words',labels,entities,label_entity,topn=55)

loaded 99.996193%

In [32]:
lables_dict=  [l for l in labels]
import cPickle as pickle
pickle.dump(dict([[lables_dict[i],label_ebd[i]] for i in range(len(lables_dict))]),open('../../Intermediate/BBN/labels.lst','w'))

In [25]:
save2bin(label_ebd,[l for l in labels],'../../Intermediate/BBN/labels.bin')


In [26]:
model =Word2Vec.load_word2vec_format('../../bow2.words',binary=False)

# Verify if popularity prior is useful 

In [32]:
query = 'york'
for w in 

TypeError: similarity() got an unexpected keyword argument 'positive'

In [4]:
from collections import Counter
prep = set(['in','at','of','with','on'])
def head(phrase):
    splited = phrase.split()
    head = ""
    for w in splited:
        if w in prep:
            break
        head = w
    if head =="": head = splited[-1]
    return head
def conjuncts(sent):
    indices =  [(d['dep'],d['gov']) for d in sent['dep'] if d['type'] == "conj_and" or d['type'] == "conj_or"]
    return indices
def mention_conj_pairs(mentions,deps):
    results = []
    for d in deps:
        results.append([[],[]])
        for m in mentions:
            if d[0]<m.end and d[0] >=m.start:
                results[-1][0].append(m)
            if d[1]<m.end and d[1] >=m.start:
                results[-1][1].append(m)
    return [ tuple([m1,m2]) for r in results for m1 in r[0] for m2 in r[1] if m1 != m2 ]

def group_mention_by_heads(fn):
    files = dict()
    with open(fn,'r') as f:
        for line in f:
            sent = json.loads(line.strip('\r\n'))
            mentions = [Mention(int(m['start']), int(m['end']), m['labels']," ".join(sent['tokens'][m['start']:m['end']]),\
                                    mid='%s_%d_%d_%d'%(sent['fileid'], sent['senid'], m['start'], m['end'])) for m in sent['mentions']]
            if sent['fileid'] not in files: 
                files[sent['fileid']]=mentions
            else:
                files[sent['fileid']].extend(mentions)
    heads_list=[]
    for k,f in files.iteritems():
        cnts = Counter()
        heads = dict()
        for m in f:
            cnts.update(m.labels)
            h= head(m.name)
            if h not in heads:heads[h]= []
            heads[h].append(m)
        if heads!=[]:
            heads_list.append(heads)
    return heads_list

In [5]:
filename = '/Users/mayk/working/figer/baseline/PLE/Intermediate/BBN/train_new.json'
heads = group_mention_by_heads(filename)

In [6]:
def get_all_conjunctions(fn):
    with open(fn,'r') as f:
            cnt = 0
            total =0
            conjs = []
            for line in f:
                sent = json.loads(line.strip('\r\n'))
                conjunc = conjuncts(sent)
                mentions = [Mention(int(m['start']), int(m['end']), m['labels']," ".join(sent['tokens'][m['start']:m['end']]),\
                                    mid='%s_%d_%d_%d'%(sent['fileid'], sent['senid'], m['start'], m['end'])) for m in sent['mentions']]
                pairs = mention_conj_pairs(mentions,conjunc)
                if pairs != []:
                    conjs.extend(pairs)
    return conjs
def group_mention_link_by_files(fn):
    files = dict()
    conj_files = dict()
    with open(fn,'r') as f:
        for line in f:
            sent = json.loads(line.strip('\r\n'))
            conjunc = conjuncts(sent)

            mentions = [Mention(int(m['start']), int(m['end']), m['labels']," ".join(sent['tokens'][m['start']:m['end']]),\
                                    mid='%s_%d_%d_%d'%(sent['fileid'], sent['senid'], m['start'], m['end'])) for m in sent['mentions']]
            pairs = mention_conj_pairs(mentions,conjunc)

            if sent['fileid'] not in files: 
                files[sent['fileid']]=mentions
            else:
                files[sent['fileid']].extend(mentions)
            if sent['fileid'] not in conj_files: 
                conj_files[sent['fileid']]=pairs
            else:
                conj_files[sent['fileid']].extend(pairs)
    for k,f in files.iteritems():
        heads = dict()
        for m in f:
            h= head(m.name)
            if h not in heads:heads[h]= []
            heads[h].append(m)
        if len(heads)>0:
            pairs = [exauhst_pairs_from_list(len(lst),lst)for lst in heads.values() if len(lst)>=2]
            files[k]= [ p for lst in pairs for p in lst ]
            files[k].extend(conj_files[k])
    return files

In [ ]:
conjs = get_all_conjunctions(filename)
cnt=0
for p in conjs:
    if p[0].labels != p[1].labels:
        cnt+=1
        print p[0].name,p[0].labels,p[1].name,p[1].labels

In [ ]:
len(conjs),cnt

In [23]:
def read_tid(fn,delim=" "):
    with open(fn,'r') as types:
        tid = dict()
        for ln in types:
            split = ln.rstrip().split(delim)
            tid[split[0]] = int(split[1])
    return tid
def exauhst_pairs_from_list(offset,lst):
    results = []
    cur = lst[-offset]
    if offset == 2:
        return [tuple([cur,lst[-1]])]
    else:
        results.extend([tuple([cur,c]) for c in lst[-offset+1:]])
        results.extend(exauhst_pairs_from_list(offset-1,lst))
    return results

In [24]:
tid = read_tid('../../Intermediate/BBN/mention.txt')

In [8]:
byfile = group_mention_link_by_files(filename)

In [34]:
with open('../../Intermediate/BBN/relation.txt','w') as out:
    for k,lst in byfile.iteritems():
        for p in lst:
            if p[0].labels!=p[1].labels:
                intersect = [l for l in p[0].labels if l in p[1].labels]
                print p[0].name,p[0].mid,tid[p[0].mid]
                #out.write("#".join([p[0].mid,p[0].name,p[1].mid,p[1].name])+" "+" ".join(intersect)+"\n")
                out.write(" ".join([str(tid[p[0].mid]),str(tid[p[1].mid])])+" "+" ".join(intersect)+"\n")
                

Southeast Asia WSJ0418_2_9_11 51473
University WSJ0413_13_43_44 51115
University WSJ0413_14_61_62 51120
State University WSJ0413_14_60_62 51124
State University WSJ0413_14_60_62 51124
State University WSJ0413_14_60_62 51124
Pennsylvania WSJ0414_17_12_13 51269
Pennsylvania WSJ0414_17_12_13 51269
Burt Sugarman WSJ0070_3_10_12 14321
wines WSJ0071_5_16_17 14345
sweet wines WSJ0071_6_44_46 14350
sweet wines WSJ0071_6_44_46 14350
sweet wines WSJ0071_6_44_46 14350
sweet wines WSJ0071_6_44_46 14350
sweet wines WSJ0071_6_44_46 14350
sweet wines WSJ0071_6_44_46 14350
sweet wines WSJ0071_6_44_46 14350
sweet wines WSJ0071_6_44_46 14350
sweet wines WSJ0071_6_44_46 14350
sweet wines WSJ0071_6_44_46 14350
sweet wines WSJ0071_6_44_46 14350
sweet wines WSJ0071_6_44_46 14350
Terrace WSJ0071_42_11_12 14447
oranges WSJ0306_2_15_16 51764
Baltimore Sun WSJ0309_24_4_6 51858
Siemens AG WSJ1324_0_3_5 71077
AEG WSJ1324_2_0_1 71082
London WSJ0097_1_2_3 35241
Sachs WSJ0090_33_32_33 35003
Sachs WSJ0090_33_32_33 35

In [ ]:
for p in byfile['WSJ0091']:
        if p[0].name!=p[1].name:
            print p[0].mid,p[1].mid, p[0].name,"#",p[1].name#,p[0].labels,p[1].labels

In [28]:
tid.keys()

['WSJ1903_2_3_4',
 'WSJ1285_43_0_1',
 'WSJ1037_2_15_18',
 'WSJ2055_15_14_15',
 'WSJ0044_36_13_14',
 'WSJ2013_143_1_2',
 'WSJ0655_14_42_43',
 'WSJ1475_2_38_39',
 'WSJ1634_127_0_1',
 'WSJ0465_5_28_29',
 'WSJ1387_27_9_10',
 'WSJ2036_25_1_2',
 'WSJ2383_5_16_17',
 'WSJ1055_23_9_10',
 'WSJ0157_1_9_10',
 'WSJ1810_7_13_14',
 'WSJ0596_35_8_9',
 'WSJ1216_29_17_18',
 'WSJ0101_3_3_4',
 'WSJ0242_0_2_3',
 'WSJ0297_41_5_7',
 'WSJ0776_25_14_15',
 'WSJ2064_34_2_3',
 'WSJ1625_17_0_1',
 'WSJ2264_9_11_12',
 'WSJ0441_10_0_2',
 'WSJ0688_7_0_1',
 'WSJ1377_71_1_2',
 'WSJ2102_34_26_27',
 'WSJ1829_9_19_21',
 'WSJ2124_28_7_8',
 'WSJ0071_29_13_14',
 'WSJ2306_10_10_11',
 'WSJ1057_97_28_29',
 'WSJ1226_2_16_19',
 'WSJ1226_2_16_18',
 'WSJ0414_13_37_39',
 'WSJ1161_9_28_29',
 'WSJ0692_19_17_18',
 'WSJ1569_87_33_35',
 'WSJ1226_2_16_17',
 'WSJ2415_45_3_5',
 'WSJ0753_2_3_4',
 'WSJ1307_7_6_7',
 'WSJ0633_78_17_18',
 'WSJ1915_21_10_11',
 'WSJ2396_15_25_26',
 'WSJ1970_9_9_10',
 'WSJ2053_29_37_38',
 'WSJ1302_52_1_2',
 'WSJ1643